In [420]:
# Required Libraries
# environment todelete
import numpy as np
import pandas as pd
import html5lib
import matplotlib.pyplot as plt
import seaborn as sns

# Settings
sns.set_style('whitegrid')
%matplotlib inline

# Get the Data
#pf_crew = pd.read_html('https://www.encyclopedia-titanica.org/titanic-crew-list/') 
#pf_crew2 = pd.read_html('https://www.encyclopedia-titanica.org/titanic-crew-list/')[0]

from bs4 import BeautifulSoup
import urllib3

http = urllib3.PoolManager()

url = 'https://www.encyclopedia-titanica.org/titanic-crew-list/'
response = http.request('GET', url)
#soup = BeautifulSoup(response.data)
soup = BeautifulSoup(response.data, "lxml")

C:\Users\Diallo\Anaconda2\envs\todelete\lib\site-packages\urllib3\connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [421]:
# print content
#print soup

In [422]:
columns = ['Name','Age','LastName']
newDF = pd.DataFrame(columns=columns) #creates a new dataframe that's empty

In [423]:
# scrape Nome, Cognome, Email.
# actual table: Name	Age	Class/Dept	Ticket	Joined	Job	Boat [Body]	
count = 0
for tr in soup.find_all('tr')[2:]:
    tds = tr.find_all('td')
    dic = {}
    #print "Name: %s, Age: %s, Ticket: %s, Joined: %s" % \
    #      (tds[0].text, tds[1].text, tds[3].text,tds[4])
    #newDF.append({'Name':tds[0].text,'Age':tds[1].text}, ignore_index = True) 
    # force age into numeric
    dico = {'Name':tds[0].text.strip(),'Age':pd.to_numeric(tds[1].text,errors='coerce')}
    #newDF.append(dico, ignore_index = True)
    #newDF.iloc[count, 0] = 
    #print dico['Name']
    
    pattern = ' '
    string = dico['Name']
    m = re.split(pattern, string)
    n = len(m[0])+len(m[1])+2
    newDF.loc[count,'Name'] = m[0].capitalize()+' '+m[1]+'.'+' '+string[n:]    
    
    string = newDF.loc[count,'Name']
    m = re.search('^[a-zA-Z\']+', string)
    txt = m.group(0)
    t = txt.capitalize()
    newDF.loc[count,'LastName'] = t  
    #newDF.loc[count,'Name'] = dico['Name']
    #newDF.iloc[count,1] = 
    #print dico['Age']
    newDF.loc[count,'Age'] = dico['Age'] 
    #pd.to_numeric(df.Age, errors='coerce')
    #print dico
    # print newDF[count] 
    count +=1


In [424]:
# All doctors are male
newDF[newDF.Name.str.contains(u'Dr')]

,Name,Age,LastName
415,"Harrison, Mr. Aragõa Drummond",40,Harrison
751,"O'loughlin, Dr. William Francis Norman",63,O'loughlin
916,"Simpson, Dr. John Edward",37,Simpson


In [425]:
# Signore title missing
newDF[newDF.Name.str.contains(u'Sig.')]

,Name,Age,LastName
5,"Allaria, Sig.. Battista Antonio",22,Allaria
35,"Banfi, Sig.. Ugo",24,Banfi
60,"Basilico, Sig.. Giovanni",27,Basilico
63,"Bazzi, Sig.. Narciso",33,Bazzi
79,"Bernardi, Sig.. Battista",22,Bernardi
172,"Casali, Sig.. Giulio",32,Casali
240,"Crovella, Sig.. Paolo Luigi",17,Crovella
258,"De MARSICO,. Sig. Govanni",20,De
277,"Donati, Sig.. Italo Francesco",17,Donati
316,"Fei, Sig.. Carlo",30,Fei


In [426]:
import string
def substrings_in_string(big_string, substrings):
    for substring in substrings:
        if string.find(big_string, substring) != -1:
            return substring
    print big_string
    return np.nan

title_list=['Mrs', 'Mr', 'Master', 'Miss', 'Major', 'Rev',
                    'Dr', 'Ms', 'Mlle','Col', 'Capt', 'Mme', 'Countess',
                    'Don', 'Jonkheer','Sig.']

newDF['Title']=newDF['Name'].map(lambda x: substrings_in_string(x, title_list))
 
#replacing all titles with mr, mrs, miss, master
def replace_titles(x):
    title=x['Title']
    if title in ['Don', 'Major', 'Capt', 'Jonkheer', 'Rev', 'Col','Sig.']:
        return 'Mr'
    elif title in ['Countess', 'Mme']:
        return 'Mrs'
    elif title in ['Mlle', 'Ms']:
        return 'Miss'
    elif title =='Dr':
        return 'Mr'
    else:
        return title
    
newDF['Title']=newDF.apply(replace_titles, axis=1)

Monrós, Sr.. Joan Javier


In [427]:
newDF.head()

,Name,Age,LastName,Title
0,"Abrams, Mr. William Thomas",34,Abrams,Mr
1,"Adams, Mr. Robert John",26,Adams,Mr
2,"Ahier, Mr. Percy Snowden",20,Ahier,Mr
3,"Akerman, Mr. Albert Edward",31,Akerman,Mr
4,"Akerman, Mr. Joseph Francis",37,Akerman,Mr


In [428]:
# Signore title missing
newDF[newDF.Name.str.contains(u'Sig.')]

,Name,Age,LastName,Title
5,"Allaria, Sig.. Battista Antonio",22,Allaria,Mr
35,"Banfi, Sig.. Ugo",24,Banfi,Mr
60,"Basilico, Sig.. Giovanni",27,Basilico,Mr
63,"Bazzi, Sig.. Narciso",33,Bazzi,Mr
79,"Bernardi, Sig.. Battista",22,Bernardi,Mr
172,"Casali, Sig.. Giulio",32,Casali,Mr
240,"Crovella, Sig.. Paolo Luigi",17,Crovella,Mr
258,"De MARSICO,. Sig. Govanni",20,De,Mr
277,"Donati, Sig.. Italo Francesco",17,Donati,Mr
316,"Fei, Sig.. Carlo",30,Fei,Mr


In [429]:
# Get the Data
path = './Data/'
filename = 'titanic-data.csv'

titanic = pd.read_csv(path+filename)
print titanic.columns
print titanic.head()

Index([u'PassengerId', u'Survived', u'Pclass', u'Name', u'Sex', u'Age',
       u'SibSp', u'Parch', u'Ticket', u'Fare', u'Cabin', u'Embarked'],
      dtype='object')
   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        

In [430]:
title_list=['Mrs', 'Mr', 'Master', 'Miss', 'Major', 'Rev',
                    'Dr', 'Ms', 'Mlle','Col', 'Capt', 'Mme', 'Countess',
                    'Don', 'Jonkheer']

# create a dictionary for replacement
newtitle = []
dicoTitle = {}
for word in title_list:
    txt = ', '+word+' '
    dicoTitle[txt] = ', '+word+'.'+' '
    
print dicoTitle

{', Capt ': ', Capt. ', ', Dr ': ', Dr. ', ', Countess ': ', Countess. ', ', Mrs ': ', Mrs. ', ', Rev ': ', Rev. ', ', Jonkheer ': ', Jonkheer. ', ', Mme ': ', Mme. ', ', Major ': ', Major. ', ', Master ': ', Master. ', ', Ms ': ', Ms. ', ', Mr ': ', Mr. ', ', Col ': ', Col. ', ', Miss ': ', Miss. ', ', Don ': ', Don. ', ', Mlle ': ', Mlle. '}


In [431]:
# loop in dictionary
for key in dicoTitle:
    print key
    #print dicoTitle[key]

, Capt 
, Dr 
, Countess 
, Mrs 
, Rev 
, Jonkheer 
, Mme 
, Major 
, Master 
, Ms 
, Mr 
, Col 
, Miss 
, Don 
, Mlle 


In [432]:
def combine_dfs(df1, df2):
    '''
    Fill in this function to take 2 DataFrames, one with subway data and one with weather data,
    and return a single dataframe with one row for each date, hour, and location. Only include
    times and locations that have both subway data and weather data available.
    '''
    s1 = pd.merge(df1, df2, how='inner', on=['Name'])
    return s1

In [433]:
s2 = combine_dfs(newDF,titanic)
print s2

               Name Age_x LastName Title  PassengerId  Survived  Pclass   Sex  \
0  Kelly, Mr. James    44    Kelly    Mr          697         0       3  male   

   Age_y  SibSp  Parch  Ticket  Fare Cabin Embarked  
0   44.0      0      0  363592  8.05   NaN        S  


In [434]:
# All doctors are male
newDF[newDF.Name.str.contains(u'Kelly')]

,Name,Age,LastName,Title
528,"Kelly, Mr. James",44,Kelly,Mr
529,"Kelly, Mr. William",23,Kelly,Mr


In [435]:
# All doctors are male
titanic[titanic.Name.str.contains(u'Kelly')]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
300,301,1,3,"Kelly, Miss. Anna Katherine ""Annie Kate""",female,NaN,0,0,9234,7.75,NaN,Q
573,574,1,3,"Kelly, Miss. Mary",female,NaN,0,0,14312,7.75,NaN,Q
696,697,0,3,"Kelly, Mr. James",male,44.0,0,0,363592,8.05,NaN,S
706,707,1,2,"Kelly, Mrs. Florence ""Fannie""",female,45.0,0,0,223596,13.50,NaN,S


In [436]:
path = 'C:/Users/Diallo/Downloads/'
filename2 = 'testTitanic.csv'
test = pd.read_csv(path+filename2)
# All doctors are male
s2 = combine_dfs(test,newDF)
print s2

   PassengerId  Pclass              Name   Sex  Age_x  SibSp  Parch  Ticket  \
0          892       3  Kelly, Mr. James  male   34.5      0      0  330911   
1         1008       3  Thomas, Mr. John  male    NaN      0      0    2681   

     Fare Cabin Embarked Age_y LastName Title  
0  7.8292   NaN        Q    44    Kelly    Mr  
1  6.4375   NaN        C    25   Thomas    Mr  


In [437]:
for ix in titanic.index:
    #print ix
    string = titanic.loc[ix,'Name']
    m = re.search('^[a-zA-Z\']+', string)
    txt = m.group(0)
    t = txt.capitalize()
    titanic.loc[ix,'LastName'] = t  


In [443]:
for ix in test.index:
    #print ix
    string = test.loc[ix,'Name']
    m = re.search('^[a-zA-Z\']+', string)
    txt = m.group(0)
    t = txt.capitalize()
    test.loc[ix,'LastName'] = t 

In [438]:
def combine_dfs2(df1, df2):
    '''
    Fill in this function to take 2 DataFrames, one with subway data and one with weather data,
    and return a single dataframe with one row for each date, hour, and location. Only include
    times and locations that have both subway data and weather data available.
    '''
    s1 = pd.merge(df1, df2, how='inner', on=['LastName'])
    return s1

In [439]:
s2 = combine_dfs2(newDF,titanic)
print s2[s2.Fare <1]

                            Name_x Age_x    LastName Title  PassengerId  \
12     Andrews, Mr. Charles Edward    19     Andrews    Mr          807   
14          Andrews, Mr. Archibald    38     Andrews    Mr          807   
51     Campbell, Mr. Donald Samuel    28    Campbell    Mr          467   
62         Cunningham, Mr. Bernard    35  Cunningham    Mr          414   
63      Cunningham, Mr. Andrew Orr    38  Cunningham    Mr          414   
134        Harrison, Mr. Norman E.    38    Harrison    Mr          264   
135  Harrison, Mr. Aragõa Drummond    40    Harrison    Mr          264   
161             Knight, Mr. George    49      Knight    Mr          733   
162     Knight, Mr. Leonard George    21      Knight    Mr          733   
164           Leonard, Mr. Matthew    25     Leonard    Mr          180   
313             Watson, Mr. Willim    27      Watson    Mr          675   
314     Watson, Mr. William Albert    15      Watson    Mr          675   

     Survived  Pclass   

In [440]:
titanic[titanic.Name.str.contains(u'Leonard')]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,LastName
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S,Palsson
121,122,0,3,"Moore, Mr. Leonard Charles",male,NaN,0,0,A4. 54510,8.0500,NaN,S,Moore
179,180,0,3,"Leonard, Mr. Lionel",male,36.0,0,0,LINE,0.0000,NaN,S,Leonard
248,249,1,1,"Beckwith, Mr. Richard Leonard",male,37.0,1,1,11751,52.5542,D35,S,Beckwith
386,387,0,3,"Goodwin, Master. Sidney Leonard",male,1.0,5,2,CA 2144,46.9000,NaN,S,Goodwin
626,627,0,2,"Kirkland, Rev. Charles Leonard",male,57.0,0,0,219533,12.3500,NaN,Q,Kirkland
655,656,0,2,"Hickman, Mr. Leonard Mark",male,24.0,2,0,S.O.C. 14879,73.5000,NaN,S,Hickman
758,759,0,3,"Theobald, Mr. Thomas Leonard",male,34.0,0,0,363294,8.0500,NaN,S,Theobald
871,872,1,1,"Beckwith, Mrs. Richard Leonard (Sallie Monypeny)",female,47.0,1,1,11751,52.5542,D35,S,Beckwith


In [441]:
newDF[newDF.Name.str.contains(u'Leonard')]

,Name,Age,LastName,Title
449,"Hoare, Mr. Leonard James",19,Hoare,Mr
452,"Hodgkinson, Mr. Leonard",46,Hodgkinson,Mr
554,"Knight, Mr. Leonard George",21,Knight,Mr
569,"Leonard, Mr. Matthew",25,Leonard,Mr
984,"Taylor, Mr. Leonard",19,Taylor,Mr
1062,"White, Mr. Leonard Lisle Oliver",32,White,Mr
1065,"White, Mr. Frank Leonard",28,White,Mr


In [444]:
s2 = combine_dfs2(newDF,test)
print s2[s2.Fare <1]

Empty DataFrame
Columns: [Name_x, Age_x, LastName, Title, PassengerId, Pclass, Name_y, Sex, Age_y, SibSp, Parch, Ticket, Fare, Cabin, Embarked]
Index: []


In [446]:
titanic[titanic.Fare <1]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,LastName
179,180,0,3,"Leonard, Mr. Lionel",male,36.0,0,0,LINE,0.0,NaN,S,Leonard
263,264,0,1,"Harrison, Mr. William",male,40.0,0,0,112059,0.0,B94,S,Harrison
271,272,1,3,"Tornquist, Mr. William Henry",male,25.0,0,0,LINE,0.0,NaN,S,Tornquist
277,278,0,2,"Parkes, Mr. Francis ""Frank""",male,NaN,0,0,239853,0.0,NaN,S,Parkes
302,303,0,3,"Johnson, Mr. William Cahoone Jr",male,19.0,0,0,LINE,0.0,NaN,S,Johnson
413,414,0,2,"Cunningham, Mr. Alfred Fleming",male,NaN,0,0,239853,0.0,NaN,S,Cunningham
466,467,0,2,"Campbell, Mr. William",male,NaN,0,0,239853,0.0,NaN,S,Campbell
481,482,0,2,"Frost, Mr. Anthony Wood ""Archie""",male,NaN,0,0,239854,0.0,NaN,S,Frost
597,598,0,3,"Johnson, Mr. Alfred",male,49.0,0,0,LINE,0.0,NaN,S,Johnson
633,634,0,1,"Parr, Mr. William Henry Marsh",male,NaN,0,0,112052,0.0,NaN,S,Parr
